## Combines 2019 and 2015-2018 Pitch Data

Requirements - 2 CSV Files (in the same working directory):
1. 2019_pitches.csv
2. pitches.csv

Output: A master CSV in the same directory

Notes: We are missing Base Runner data for 2019, which we would need when we expand the state

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '../../../RawData/pitches.csv'
path_2019 = '../../../RawData/2019_pitches.csv'

In [3]:
p = pd.read_csv(path)
p_2019 = pd.read_csv(path_2019)

In [4]:
p['pitch_type'].value_counts()

FF    1014877
SL     450578
FT     337983
CH     292789
SI     242504
CU     234391
FC     149756
KC      66484
FS      43705
KN      11260
IN       6197
EP        815
FO        810
PO        628
SC        113
UN         57
AB          9
FA          9
Name: pitch_type, dtype: int64

In [5]:
#rename COLS pz to py 
p_2019.rename(columns={'pz':'py'}, inplace=True)
p.rename(columns={'pz':'py'}, inplace=True)

In [6]:
#rename SI to FT
p['pitch_type'].replace({"SI":"FT"}, inplace=True)
p_2019['pitch_type'].replace({"SI":"FT"}, inplace=True)

# replacing missing data with NaN
p_2019['on_1b'] = np.nan
p_2019['on_2b'] = np.nan
p_2019['on_3b'] = np.nan

In [7]:
print(p_2019.info())
print(p.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728790 entries, 0 to 728789
Data columns (total 40 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   px               722161 non-null  float64
 1   py               722161 non-null  float64
 2   start_speed      722161 non-null  float64
 3   end_speed        722161 non-null  float64
 4   spin_rate        722161 non-null  object 
 5   spin_dir         722161 non-null  object 
 6   break_angle      722161 non-null  float64
 7   break_length     722161 non-null  float64
 8   break_y          722161 non-null  float64
 9   ax               722161 non-null  float64
 10  ay               722161 non-null  float64
 11  az               722161 non-null  float64
 12  sz_bot           728790 non-null  float64
 13  sz_top           728790 non-null  float64
 14  type_confidence  722161 non-null  object 
 15  vx0              722161 non-null  float64
 16  vy0              722161 non-null  floa

In [8]:
keep_cols = ['ab_id','px','py','code','pitch_type','batter_id', 'type']
state_cols = ['outs', 'pitch_num', 'on_1b', 'on_2b', 'on_3b', 's_count', 'b_count']
all_cols = keep_cols + state_cols

In [9]:
p.drop(columns=[col for col in p if col not in all_cols], inplace=True)
p_2019.drop(columns=[col for col in p_2019 if col not in all_cols], inplace=True)

In [10]:
p = p.append(p_2019, ignore_index=True)

In [11]:
p.shape

(3595944, 13)

In [12]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,0.416,2.963,C,S,FF,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.191,2.347,S,S,FF,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,-0.518,3.284,F,S,FF,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,-0.641,1.221,B,B,FF,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0
4,-1.821,2.083,B,B,CU,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0


In [13]:
# delete where px does not exist
# delete where b_count is 4
p.dropna(subset=['px'], inplace=True)
p.dropna(subset=['py'], inplace=True)
p.dropna(subset=['code'], inplace=True)
p.dropna(subset=['pitch_type'], inplace=True)
p.dropna(subset=['ab_id'], inplace=True)

In [14]:
#filter only the pitches that we want
pitches = ["FF", "FT", "CU", "CH", "FC", "SL"]
filt = p['pitch_type'].isin(pitches)
p = p.loc[filt]

In [15]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,0.416,2.963,C,S,FF,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.191,2.347,S,S,FF,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,-0.518,3.284,F,S,FF,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,-0.641,1.221,B,B,FF,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0
4,-1.821,2.083,B,B,CU,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0


In [16]:
#updating ab_id dtype
p['ab_id'] = p['ab_id'].astype(np.int64)

#updating b_count dtype
p['b_count'] = p['b_count'].astype(np.int64)

#updating s_count dtype
p['s_count'] = p['s_count'].astype(np.int64)

#updating outs dtype
p['outs'] = p['outs'].astype(np.int64)

#updating pitch_num dtype
p['pitch_num'] = p['pitch_num'].astype(np.int64)

In [17]:
p.shape

(3413438, 13)

In [18]:
#making sure we show that we dont have on_base data for 2019
p.tail()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
3595939,0.30,1.99,X,X,FF,2019185244,2,0,1,4,NaN,NaN,NaN
3595940,1.00,-0.38,B,B,SL,2019185245,0,0,2,1,NaN,NaN,NaN
3595941,0.36,2.02,C,C,FF,2019185245,1,0,2,2,NaN,NaN,NaN
3595942,-0.26,2.60,C,C,SL,2019185245,1,0,2,3,NaN,NaN,NaN
3595943,0.22,1.06,S,S,SL,2019185245,1,0,2,4,NaN,NaN,NaN


### Adding Result and Swing Columns to Pitches using AtBat Data

In [19]:
path_ab = '../../../RawData/all_atbats.csv'

In [20]:
ab = pd.read_csv(path_ab)

In [21]:
ab.head()

,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand
0,2015000001,572761,Groundout,201500001,1.0,1,0.0,L,452657,L
1,2015000002,518792,Double,201500001,1.0,1,0.0,L,452657,L
2,2015000003,407812,Single,201500001,1.0,1,0.0,L,452657,R
3,2015000004,425509,Strikeout,201500001,1.0,2,0.0,L,452657,R
4,2015000005,571431,Strikeout,201500001,1.0,3,0.0,L,452657,L


In [22]:
# adding columns
p['swing'] = -1
p['res'] = ''

In [23]:
s, o, h, f,  = "strike", "out", "hit", "foul"
b, ts = "ball", "take strike"

#pitch keys
swing_strike_not_in_play = ['S', 'T', 'L', 'W', 'M', 'Q']
swing_foul_not_in_play = ['F', 'R']
take_ball = ['B', '*B']
intentional_ball = ['P', 'I']

#event keys
gets_out = ['Groundout', 'Flyout', 'Lineout', 'Forceout', 'Pop Out', 'Grounded Into DP', 'Double Play', 
            'Fielders Choice Out', 'Bunt Pop Out', 'Bunt Lineout', 'Triple Play', 'Bunt Groundout', 'Runner Out',
            'Fielders Choice', 'Strikeout - DP', 'Batter Interference']
gets_hit = ['Single', 'Double', 'Triple', 'Home Run']
sac_hit = ['Sac Fly', 'Sac Bunt', 'Sac Fly DP', 'Sacrifice Bunt DP', 'Sac Bunt Double Play']
error = ['Field Error', 'Catcher Interference', 'Fan Interference']
        
for i, row in p.iterrows():
    # batter swings and ball is NOT in play
    if row.code in swing_strike_not_in_play:
        p.at[i, 'swing'] = 1
        p.at[i, 'res'] = s
        
    # batter swings and ball is NOT in play
    elif row.code in swing_foul_not_in_play:
        p.at[i, 'swing'] = 1
        p.at[i, 'res'] = f
        
    # batter takes and its a ball
    elif row.code in take_ball:
        p.at[i, 'swing'] = 0
        p.at[i, 'res'] = b
        
    # pitcher intentionally throws a ball to the batter
    elif row.code in intentional_ball: continue

    # takes a strike
    elif row.code == 'C':
        p.at[i, 'swing'] = 0
        p.at[i, 'res'] = s
        
    # batter takes but gets on base (hit by pitch)
    elif row.code == 'H': continue
        
    # this contains all rows in which code does not exist and is NaN
    elif row.code != row.code: continue
        
    elif row.type == 'X':
        filt = ab['ab_id'] == row['ab_id']
        res = ab.loc[filt, 'event'].values[0]

        # gets out swinging
        if res in gets_out:
            p.at[i, 'swing'] = 1
            p.at[i, 'res'] = o
        
        # gets on base swinging
        elif res in gets_hit:
            p.at[i, 'swing'] = 1
            p.at[i, 'res'] = h
            
        # batter is not trying to get on base
        elif res in sac_hit: continue
            
        # fielding team makes an error
        elif res in error: continue
           
        # result is unkown
        else:
            print(res)

In [24]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res
0,0.416,2.963,C,S,FF,2015000001,0,0,0,1,0.0,0.0,0.0,0,strike
1,-0.191,2.347,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike
2,-0.518,3.284,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul
3,-0.641,1.221,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball
4,-1.821,2.083,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball


In [25]:
#filtering/dropping columns with swing = -1, that means there was a continue triggered from before
filt = p['swing'] != -1
p = p.loc[filt]

In [26]:
p['swing'].value_counts(normalize=True)

0    0.537535
1    0.462465
Name: swing, dtype: float64

In [27]:
print(p['res'].value_counts(normalize=True))

ball      0.365378
strike    0.289854
foul      0.179948
out       0.116813
hit       0.048006
Name: res, dtype: float64


In [28]:
print(p.shape)

(3346063, 15)


In [29]:
filt = p['swing'] == 1

p_swing = p.loc[filt]
p_take = p.loc[~filt]

In [30]:
print(p_swing.shape)
print(p_take.shape)

(1547437, 15)
(1798626, 15)


In [31]:
p_swing['res'].value_counts(normalize=True)

foul      0.389107
strike    0.254500
out       0.252589
hit       0.103804
Name: res, dtype: float64

In [32]:
p_take['res'].value_counts(normalize=True)

ball      0.679729
strike    0.320271
Name: res, dtype: float64

In [33]:
p.to_csv('./all_pitches_results.csv', index=False)

In [34]:
testdf = pd.read_csv('./all_pitches_results.csv')

In [35]:
testdf.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res
0,0.416,2.963,C,S,FF,2015000001,0,0,0,1,0.0,0.0,0.0,0,strike
1,-0.191,2.347,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike
2,-0.518,3.284,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul
3,-0.641,1.221,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball
4,-1.821,2.083,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball
